In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# Step 1
# Input maximum gm/id for input transistors

ID = 5e-6
BW = 1e3
Av = 80 # dB
CL = 200e-15

# Convert Av to gain
Av = 10**(Av/20)

gmid_1 = 25
gm_1 = gmid_1*ID/2

def sci(num):
    return "{:.2e}".format(num)

Rout = Av/gm_1
GBW = gm_1 / CL

print("gm1: {}".format(sci(gm_1)))
print("required ROUT: {}".format(sci(Rout)))
print("GBW: {}".format(sci(GBW)))

gm1: 6.25e-05
required ROUT: 1.60e+08
GBW: 3.12e+08


In [32]:
# Step 2
# Rout is given by ro6 || r08
# Must fulfill Av both also poles

# Dominant
f1 = 1/(2*np.pi*Rout*CL)
print("Dominant pole frequency: {}".format(sci(f1)))

# Non-Dominant poles MUST be high enough
# f2 = gm3 / C3
# f3 = gm3 / C3
# f4 = gm7 / C7
f_non = 20*np.log10(Av) / 20 * f1 * 10
print("required non-dominant pole frequency: {}".format(sci(f_non)))
# fint ft

Dominant pole frequency: 4.97e+03
required non-dominant pole frequency: 1.99e+05


In [33]:
# Step 3
# Find possible gm/id, gmro and ft

cascoded = True

gmid_3 = 5
gm_3 = gmid_3*ID/2
gmro_3 = 1.17e1
ft_3 = 7.3e7
W_3 = 1.14e-6
L_3 = 5e-6

gmid_cascn = 20
gm_cascn = gmid_cascn*ID/2
gmro_cascn = 8.01e2
ft_cascn = 4.54e5
W_cascn = 3.15e-4
L_cascn = 20e-6

gmid_cascp = 20
gm_cascp = gmid_cascp*ID/2
gmro_cascp = 1.72e3
ft_cascp = 2.41e5
W_cascp = 7.29e-4
L_cascp = 20e-6

gmid_7 = 5
gm_7 = gmid_7*ID/2
gmro_7 = 1.29e1
ft_7 = 2.27e7
W_7 = 4.14e-6
L_7 = 5e-6

def parallel(a, b):
    return a*b/(a+b)

# In a cascode stage M2 boosts the output impedance of M1
# by a facor of gm2*ro2
def cascode_rout(ro1, gm2, ro2):
    return ro1 * gm2 * ro2

def db(val):
    return np.round(20*np.log10(val), 2)

ro3 = gmro_3/gm_3
ro7 = gmro_7/gm_7
ro_cascn = gmro_cascn/gm_cascn
ro_cascp = gmro_cascp/gm_cascp
ro_n = cascode_rout(ro3, gm_cascn, ro_cascn)
ro_p = cascode_rout(ro3, gm_cascn, ro_cascn)

print("ro_n: {}".format(sci(ro_n)))
print("ro_p: {}".format(sci(ro_p)))

Rout_calc = parallel(ro_n, ro_p)

if Rout_calc < Rout:
    print("ro3||ro7 = {0} LOWER than {1}".format(sci(Rout_calc), sci(Rout)))
else:
    print("ro3||ro7 = {0} HIGHER than {1}".format(sci(Rout_calc), sci(Rout)))

Av_new = db(Rout_calc*gm_1)
print("Actual gain: {} dB".format(Av_new))

ro_n: 7.50e+08
ro_p: 7.50e+08
ro3||ro7 = 3.75e+08 HIGHER than 1.60e+08
Actual gain: 87.4 dB


In [34]:
# Step 4
# POLES

# Determine actual CL from ft

# Non-Dominant poles MUST be high enough
# f2 = gm3 / C3
# f3 = gm3 / C3
# f4 = gm7 / C7

cgg_3 = gm_3 / (2*np.pi*ft_3)
cgg_7 = gm_7 / (2*np.pi*ft_7)
cgg_cascn = gm_cascn / (2*np.pi*ft_cascn)
cgg_cascp = gm_cascp / (2*np.pi*ft_cascp)

# Estimate all capacitances
caps_3 = {
    "cgg": 0.0,
    "cgd": 0.0,
    "cdd": 0.0,
    "cdb": 0.0,
}
caps_7 = {
    "cgg": 0.0,
    "cgd": 0.0,
    "cdd": 0.0,
    "cdb": 0.0,
}
caps_cascn = {
    "cgg": 0.0,
    "cgd": 0.0,
    "cdd": 0.0,
    "cdb": 0.0,
}
caps_cascp = {
    "cgg": 0.0,
    "cgd": 0.0,
    "cdd": 0.0,
    "cdb": 0.0,
}
def estimate_caps(caps, cgg):
    for cap in caps:
        match cap:
            case "cgg":
                caps[cap] = cgg
            case "cgd":
                caps[cap] = 0.24*cgg
            case "cdd":
                caps[cap] = 0.6*cgg
            case "cdb":
                caps[cap] = caps["cdd"] - caps["cgd"]
    return caps
                
caps_3 = estimate_caps(caps_3, cgg_3)
caps_7 = estimate_caps(caps_7, cgg_7)
caps_cascn = estimate_caps(caps_cascn, cgg_cascn)
caps_cascp = estimate_caps(caps_cascp, cgg_cascp)

# Add parasitic output capacitances
CL_new = CL + (caps_cascn["cgd"]/2) + (caps_cascp["cgd"]/2)

# New dominant pole
f1_new = 1/(2*np.pi*Rout_calc*CL_new)
print("Dominant pole frequency: {}".format(sci(f1_new)))

# Extrapolate from dominate pole the approximate 
# minimum frequency for non-dominant
f_non_new = f1_new * 10 **(Av_new / 20)
print("required non-dominant pole frequency: {}".format(sci(f_non_new)))

# Determine gain at 1k from dominant pole
decades_to_bww = np.log10(Av_new/f1_new)
print("Gain at 1k: {} dB".format(np.round(Av_new - np.abs(20*decades_to_bww), 2)))

# Calculate non-dominant pole frequency
# Remember there are 2 poles from nmos mirrors
f_non_dom = gm_3 / (2*caps_3["cgg"]*2*np.pi)
print("fp nmirror: {}".format(sci(f_non_dom)))

f_non_dom_p = gm_7 / (2*caps_7["cgg"]*2*np.pi)
print("fp pmirror: {}".format(sci(f_non_dom_p)))

# Maximum C
max_C3 = gm_3 / (2*np.pi*f_non_new)
max_C7 = gm_7 / (2*np.pi*f_non_new)
print("Max C3: {}".format(sci(max_C3)))
print("Max C7: {}".format(sci(max_C7)))

GBW_calc = gm_1 / CL_new
print("GBW: {}".format(sci(GBW_calc)))

Dominant pole frequency: 6.78e+01
required non-dominant pole frequency: 1.59e+06
Gain at 1k: 85.19 dB
fp nmirror: 3.65e+07
fp pmirror: 1.14e+07
Max C3: 1.25e-12
Max C7: 1.25e-12
GBW: 9.97e+06


In [20]:
gm_3

7.999999999999999e-05

In [19]:
caps_3

{'cgg': 1.5955382766104797e-11,
 'cgd': 3.829291863865151e-12,
 'cdd': 9.573229659662877e-12,
 'cdb': 5.743937795797726e-12}

In [27]:
gm_3

3e-05

In [28]:
caps_3

{'cgg': 5.983268537289299e-12,
 'cgd': 1.4359844489494318e-12,
 'cdd': 3.5899611223735796e-12,
 'cdb': 2.1539766734241476e-12}